In [135]:
import torch
import torch.nn as nn
import torch.nn.functional as nf
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [145]:
train = pd.read_csv(r'C:\ML Projects\Project3\train.csv',dtype=np.float32)
test = pd.read_csv(r'C:\ML Projects\Project3\test.csv',dtype=np.float32)
submission = pd.read_csv(r'C:\ML Projects\Project3\sample_submission.csv')

In [127]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
num_epochs = 6
num_classes = 10
batch_size = 100
learning_rate = 0.01

In [129]:
y = train['label'].values
X = train.drop(['label'],1).values/255.0
X=X.reshape((-1, 1, 28, 28))
X.shape


(42000, 1, 28, 28)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape

(33600, 1, 28, 28)

In [131]:
XTrain = torch.from_numpy(X_train)
yTrain = torch.from_numpy(y_train).type(torch.LongTensor)
XTest = torch.from_numpy(X_test)
yTest = torch.from_numpy(y_test).type(torch.LongTensor)

In [132]:
train_df = TensorDataset(XTrain, yTrain)
test_df = TensorDataset(XTest, yTest)
train_loader = DataLoader(train_df, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_df, batch_size = batch_size, shuffle = False)

In [133]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

model = CNN()
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [134]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = error(outputs, labels)
        loss_list.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/6], Step [100/336], Loss: 0.4190, Accuracy: 85.00%
Epoch [1/6], Step [200/336], Loss: 0.2960, Accuracy: 91.00%
Epoch [1/6], Step [300/336], Loss: 0.4448, Accuracy: 87.00%
Epoch [2/6], Step [100/336], Loss: 0.2601, Accuracy: 91.00%
Epoch [2/6], Step [200/336], Loss: 0.4166, Accuracy: 85.00%
Epoch [2/6], Step [300/336], Loss: 0.2756, Accuracy: 90.00%
Epoch [3/6], Step [100/336], Loss: 0.3406, Accuracy: 89.00%
Epoch [3/6], Step [200/336], Loss: 0.2967, Accuracy: 92.00%
Epoch [3/6], Step [300/336], Loss: 0.3680, Accuracy: 88.00%
Epoch [4/6], Step [100/336], Loss: 0.3194, Accuracy: 92.00%
Epoch [4/6], Step [200/336], Loss: 0.1582, Accuracy: 96.00%
Epoch [4/6], Step [300/336], Loss: 0.2243, Accuracy: 94.00%
Epoch [5/6], Step [100/336], Loss: 0.7421, Accuracy: 86.00%
Epoch [5/6], Step [200/336], Loss: 0.3465, Accuracy: 90.00%
Epoch [5/6], Step [300/336], Loss: 0.3404, Accuracy: 93.00%
Epoch [6/6], Step [100/336], Loss: 0.3637, Accuracy: 91.00%
Epoch [6/6], Step [200/336], Loss: 0.199

In [148]:
Xt = (test.values/255).reshape((-1, 1, 28, 28))
Xt.shape

(28000, 1, 28, 28)

In [149]:
model.eval()
with torch.no_grad():
    submission['Label'] = model(torch.from_numpy(Xt)).argmax(dim=1)
    
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [151]:
submission.to_csv("submission2.csv", index=False)